<a href="https://colab.research.google.com/github/lomerrod/proyectofinalDS2/blob/main/ProyectoParteIII%2BRodriguezLorena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PROYECTO EMPRESA DE VENTA DE LEÑA A HOGARES Y COMERCIOS**






##**📌 Introducción**##

El presente proyecto tiene como objetivo predecir la fuga de clientes en una empresa de distribución de leña, utilizando técnicas de Machine Learning. Se entiende por "fuga" a los clientes que no han realizado compras en los últimos seis meses.

La empresa cuenta con información histórica de sus operaciones, que incluye:

*   Datos de ventas y facturación
*   Detalles de los productos vendidos
*   Datos generales de clientes
*   Resultados de encuestas de satisfacción
*   Reclamos realizados
*   Información sobre deudas

A partir de este conjunto de datos, se busca construir un modelo que pueda identificar patrones comunes entre los clientes que dejaron de comprar y anticipar futuras fugas. Esto permitirá a la empresa tomar decisiones proactivas para retener clientes, mejorar su servicio y optimizar sus estrategias comerciales.

##**1- Preparación de la información**##

###*1.1 Librerías*###

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

###*1.2 Carga de los datasets*###

In [9]:
# URL del archivo Excel en formato raw
file_url = "https://raw.githubusercontent.com/lomerrod/proyectofinalDS2/main/data.xlsx"

# Cargar el archivo
xls = pd.ExcelFile(file_url)

# Verificar los nombres de las hojas
print(xls.sheet_names)

['ventas', 'ventas_detalle', 'productos', 'cliente', 'encuesta2024', 'nc', 'deuda']


In [11]:
# Cargar las hojas
df_ventas = xls.parse('ventas')
df_ventas_detalle = xls.parse('ventas_detalle')
df_productos = xls.parse('productos')
df_cliente = xls.parse('cliente')
df_encuesta2024 = xls.parse('encuesta2024')
df_reclamos2024 = xls.parse('nc')
df_deuda = xls.parse('deuda')

##**2- Primera aproximación a los datasets**##

###*2.1 Ver las primeras filas*####

In [20]:
df_ventas.head()

,FECHA DE FACTURACION,DOCUMENTO,COMPROBANTE,CLIENTE,TOTAL,FECHA DE ENTREGA,DISTRIBUIDOR,CHOFER
0,2022-01-01,RFACEF,32595,0012585,7950.0,2022-09-29,STP5810,NaN
1,2022-01-01,RNCREDEF,6584,0010141,-9528.0,2021-12-17,FAW5,SANDOVAL
2,2022-01-01,RNCREDTK,11348,0012585,-7950.0,2022-07-23,STP5810,NaN
3,2022-01-03,RFACEF,25413,0009049,4950.0,2022-01-03,FAW5,NICOLASM
4,2022-01-04,RFACEF,25457,0012253,3795.0,2022-01-04,FAW5,NICOLASM


In [26]:
df_ventas_detalle.head()

,COMPROBANTE,CODIGO_PRODUCTO,CANTIDAD
0,32595,AS0004,1.0
1,6584,CO0027,-0.7
2,6584,AS0004,-0.3
3,11348,AS0004,-1.0
4,25413,AS0004,0.9


In [27]:
df_productos.head()

,CODIGO_PRODUCTO,PRODUCTO,UNIDAD,FAMILIA
0,AS0002,ASTILLA CALEFACCION CHICA,TON,AST
1,AS0003,ASTILLA CALEFACTOR,TON,AST
2,AS0004,ASTILLA ESTUFA,TON,AST
3,AS0005,ASTILLA GRANDE,TON,AST
4,AS0006,ASTILLON CHICO,TON,AST


In [29]:
df_cliente.head()

,CLIENTE,CLIENTE_NOMBRE,INGRESO,ZONA,TIPO DE VENTA,MEDIO,SEGMENTADOR
0,0000000,REPARTO,00:00:00,NaN,REPARTO,NaN,NaN
1,0000001,MOSTRADOR,00:00:00,NaN,MOSTRADOR,NaN,NaN
2,0000002,ARANCIBIA GONZALO,00:00:00,CARRASCO NORTE,REPARTO,NaN,CLIENTE DE REPARTO COMUN
3,0000003,ARANCIBIA BERNARDO (TATO),00:00:00,POCITOS,REPARTO,NaN,CLIENTE DE REPARTO COMUN
4,0000012,JORAJURIA GABRIELA,00:00:00,CARRASCO,REPARTO,NaN,CLIENTE DE REPARTO COMUN


In [30]:
df_encuesta2024.head()

,CLIENTE,ATENCION,CALIDAD,PLAZO_ACORDADO,CONTACTO,FACTOR_IMPORTANTE,VAL_PRECIO,VAL_CALIDAD,VAL_CANTIDAD,VAL_PUNTUALIDAD,VAL_ORDEN,VAL_PERSONAL,F_ENTREGA
0,0011420,3,3,Si,Publicidad en redes/televisión,Calidad de la leña,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01
1,0006365,3,3,Si,Búsqueda en internet,Precio,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01
2,0011707,3,3,Si,Visita a nuestras instalaciones (oficina o dep...,Precio,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01
3,0010287,3,3,Si,Búsqueda en internet,Servicio telefónico y del personal,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01
4,0012712,3,3,Si,Publicidad en redes/televisión,Calidad de la leña,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01


In [31]:
df_reclamos2024.head()

,Fecha de entrega,Fecha de reclamo,CLIENTE,CHOFER,Reclamo,Tipo de error,Nivel de disconformidad,Área afectada,Categoría,Visita Realizada,Medidas a tomar,Responsable,Fecha de cierre,Satisfaccion
0,2024-01-04,2024-01-08,11040,Tapia,NC,Error del proceso,Medio,Logística,Condiciones de entrega,No,Pedido adecuado en próxima entrega,LB,2024-01-08 00:00:00,Satisfecho
1,2024-01-08,2024-01-11,12141,Tapia,Reclamo que N/A,Error del proceso,Leve,Producción,Tipo de leña,No,No aplica,LB,2024-01-11 00:00:00,Satisfecho
2,2024-01-08,2024-01-15,13060,Nicolas,Sugerencia,Error del proceso,Medio,Producción,Tamaño de la leña,No,Leña adecuada en próximo pedido,IP,2024-01-18 00:00:00,Muy satisfecho
3,2024-01-08,2024-01-15,12526,Nicolas,NC,Error del proceso,Medio,Producción,Calidad de la leña,No,Regalo de leña Serrana,FP,2024-02-20 00:00:00,Satisfecho
4,2024-01-19,2024-01-19,12141,Tapia,NC,Error del proceso,Medio,Comercial,Tamaño de la leña,No,Nota de crédito,FP,2024-01-22 00:00:00,Muy satisfecho


In [32]:
df_deuda.head()

,CLIENTE,monto_deuda,dias_vencido
0,3831,8459,15
1,4966,14565,15
2,5342,12524,15
3,5582,21698,15
4,5752,7109,15


###*2.2 Información general de columnas y tipos de datos*####

In [21]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17595 entries, 0 to 17594
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   FECHA DE FACTURACION  17595 non-null  datetime64[ns]
 1   DOCUMENTO             17595 non-null  object        
 2   COMPROBANTE           17595 non-null  int64         
 3   CLIENTE               17595 non-null  object        
 4   TOTAL                 17595 non-null  float64       
 5   FECHA DE ENTREGA      17595 non-null  datetime64[ns]
 6   DISTRIBUIDOR          17050 non-null  object        
 7   CHOFER                9387 non-null   object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 1.1+ MB


In [22]:
df_ventas.shape

(17595, 8)

Tenemos 17595 filas y 8 columnas para el *dataframe de ventas*

In [33]:
df_ventas_detalle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31428 entries, 0 to 31427
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   COMPROBANTE      31428 non-null  int64  
 1   CODIGO_PRODUCTO  31428 non-null  object 
 2   CANTIDAD         31428 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 736.7+ KB


In [34]:
df_ventas_detalle.shape

(31428, 3)

Tenemos 31428 filas y 3 columnas para el *dataframe de ventas_detalle*

In [35]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODIGO_PRODUCTO  41 non-null     object
 1   PRODUCTO         41 non-null     object
 2   UNIDAD           41 non-null     object
 3   FAMILIA          41 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [36]:
df_productos.shape

(41, 4)

Tenemos 41 filas y 4 columnas para el *dataframe de productos*

In [37]:
df_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9786 entries, 0 to 9785
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CLIENTE         9785 non-null   object
 1   CLIENTE_NOMBRE  9786 non-null   object
 2   INGRESO         9786 non-null   object
 3   ZONA            8783 non-null   object
 4   TIPO DE VENTA   9308 non-null   object
 5   MEDIO           2372 non-null   object
 6   SEGMENTADOR     9043 non-null   object
dtypes: object(7)
memory usage: 535.3+ KB


In [38]:
df_cliente.shape

(9786, 7)

Tenemos 9786 filas y 7 columnas para el *dataframe de clientes*

In [39]:
df_encuesta2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CLIENTE            279 non-null    object        
 1   ATENCION           285 non-null    int64         
 2   CALIDAD            285 non-null    int64         
 3   PLAZO_ACORDADO     285 non-null    object        
 4   CONTACTO           285 non-null    object        
 5   FACTOR_IMPORTANTE  285 non-null    object        
 6   VAL_PRECIO         86 non-null     float64       
 7   VAL_CALIDAD        86 non-null     float64       
 8   VAL_CANTIDAD       86 non-null     float64       
 9   VAL_PUNTUALIDAD    86 non-null     float64       
 10  VAL_ORDEN          86 non-null     float64       
 11  VAL_PERSONAL       86 non-null     float64       
 12  F_ENTREGA          285 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(2), object(4)
memory usag

In [40]:
df_encuesta2024.shape

(285, 13)

Tenemos 285 filas y 13 columnas para el *dataframe de encuesta*

In [41]:
df_reclamos2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Fecha de entrega         149 non-null    datetime64[ns]
 1   Fecha de reclamo         149 non-null    datetime64[ns]
 2   CLIENTE                  149 non-null    int64         
 3   CHOFER                   149 non-null    object        
 4   Reclamo                  149 non-null    object        
 5   Tipo de error            149 non-null    object        
 6   Nivel de disconformidad  149 non-null    object        
 7   Área afectada            149 non-null    object        
 8   Categoría                149 non-null    object        
 9   Visita Realizada         138 non-null    object        
 10  Medidas a tomar          149 non-null    object        
 11  Responsable              149 non-null    object        
 12  Fecha de cierre          149 non-nul

In [42]:
df_reclamos2024.shape

(149, 14)

Tenemos 149 filas y 14 columnas para el dataframe de ventas

In [43]:
df_deuda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   CLIENTE       41 non-null     int64
 1   monto_deuda   41 non-null     int64
 2   dias_vencido  41 non-null     int64
dtypes: int64(3)
memory usage: 1.1 KB


In [44]:
df_deuda.shape

(41, 3)

Tenemos 41 filas y 3 columnas para el dataframe de ventas

###*2.3 Estadísticas numéricas básicas*####


In [23]:
df_ventas.describe()

,FECHA DE FACTURACION,COMPROBANTE,TOTAL,FECHA DE ENTREGA
count,17595,17595.000000,17595.000000,17595
mean,2023-08-25 01:52:36.828644608,42940.270247,8060.773600,2023-08-25 08:24:13.503836160
min,2022-01-01 00:00:00,6538.000000,-362200.000000,2021-03-29 00:00:00
25%,2022-09-01 00:00:00,37001.500000,4500.000000,2022-09-02 00:00:00
50%,2023-08-18 00:00:00,43346.000000,7895.000000,2023-08-18 00:00:00
75%,2024-06-22 00:00:00,55014.500000,10225.000000,2024-06-22 00:00:00
max,2025-04-30 00:00:00,60596.000000,362200.000000,2025-05-02 00:00:00
std,NaN,11949.514767,9976.423831,NaN


In [45]:
df_ventas_detalle.describe()

,COMPROBANTE,CANTIDAD
count,31428.000000,31428.000000
mean,43499.945654,1.300381
std,12057.404122,6.134941
min,6538.000000,-200.000000
25%,37993.750000,0.500000
50%,44120.000000,0.500000
75%,55390.000000,1.000000
max,60596.000000,270.000000


In [48]:
df_encuesta2024.describe()

,ATENCION,CALIDAD,VAL_PRECIO,VAL_CALIDAD,VAL_CANTIDAD,VAL_PUNTUALIDAD,VAL_ORDEN,VAL_PERSONAL,F_ENTREGA
count,285.000000,285.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,285
mean,2.824561,2.754386,3.709302,4.569767,4.593023,4.790698,4.802326,4.720930,2024-05-16 22:24:00
min,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,2024-01-01 00:00:00
25%,3.000000,3.000000,3.000000,4.000000,4.250000,5.000000,5.000000,5.000000,2024-04-01 00:00:00
50%,3.000000,3.000000,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,2024-05-01 00:00:00
75%,3.000000,3.000000,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,2024-07-01 00:00:00
max,3.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,2024-12-01 00:00:00
std,0.456672,0.477696,0.956295,0.775215,0.872831,0.511429,0.647525,0.745988,NaN


In [49]:
df_deuda.describe()

,CLIENTE,monto_deuda,dias_vencido
count,41.000000,41.000000,41.000000
mean,9784.609756,15304.682927,43.170732
std,3173.803309,13330.477562,34.726721
min,296.000000,1000.000000,15.000000
25%,6984.000000,8459.000000,15.000000
50%,10544.000000,10375.000000,15.000000
75%,12451.000000,18916.000000,90.000000
max,13451.000000,59400.000000,90.000000


###*2.4 Verificar valores nulos*####


In [24]:
df_ventas.isnull().sum()

,0
FECHA DE FACTURACION,0
DOCUMENTO,0
COMPROBANTE,0
CLIENTE,0
TOTAL,0
FECHA DE ENTREGA,0
DISTRIBUIDOR,545
CHOFER,8208


In [52]:
df_cliente.isnull().sum()

,0
CLIENTE,1
CLIENTE_NOMBRE,0
INGRESO,0
ZONA,1003
TIPO DE VENTA,478
MEDIO,7414
SEGMENTADOR,743


In [53]:
df_encuesta2024.isnull().sum()

,0
CLIENTE,6
ATENCION,0
CALIDAD,0
PLAZO_ACORDADO,0
CONTACTO,0
FACTOR_IMPORTANTE,0
VAL_PRECIO,199
VAL_CALIDAD,199
VAL_CANTIDAD,199
VAL_PUNTUALIDAD,199


In [54]:
df_reclamos2024.isnull().sum()

,0
Fecha de entrega,0
Fecha de reclamo,0
CLIENTE,0
CHOFER,0
Reclamo,0
Tipo de error,0
Nivel de disconformidad,0
Área afectada,0
Categoría,0
Visita Realizada,11


###*2.5 Nombre de las columnas*####

In [25]:
df_ventas.columns

Index(['FECHA DE FACTURACION', 'DOCUMENTO', 'COMPROBANTE', 'CLIENTE', 'TOTAL',
       'FECHA DE ENTREGA', 'DISTRIBUIDOR', 'CHOFER'],
      dtype='object')

In [56]:
df_ventas_detalle.columns

Index(['COMPROBANTE', 'CODIGO_PRODUCTO', 'CANTIDAD'], dtype='object')

In [57]:
df_productos.columns

Index(['CODIGO_PRODUCTO', 'PRODUCTO', 'UNIDAD', 'FAMILIA'], dtype='object')

In [58]:
df_cliente.columns

Index(['CLIENTE', 'CLIENTE_NOMBRE', 'INGRESO', 'ZONA', 'TIPO DE VENTA',
       'MEDIO', 'SEGMENTADOR'],
      dtype='object')

In [59]:
df_encuesta2024.columns

Index(['CLIENTE', 'ATENCION', 'CALIDAD', 'PLAZO_ACORDADO', 'CONTACTO',
       'FACTOR_IMPORTANTE', 'VAL_PRECIO', 'VAL_CALIDAD', 'VAL_CANTIDAD',
       'VAL_PUNTUALIDAD', 'VAL_ORDEN', 'VAL_PERSONAL', 'F_ENTREGA'],
      dtype='object')

In [60]:
df_reclamos2024.columns

Index(['Fecha de entrega', 'Fecha de reclamo', 'CLIENTE', 'CHOFER', 'Reclamo',
       'Tipo de error', 'Nivel de disconformidad', 'Área afectada',
       'Categoría', 'Visita Realizada', 'Medidas a tomar', 'Responsable',
       'Fecha de cierre', 'Satisfaccion'],
      dtype='object')

In [61]:
df_deuda.columns

Index(['CLIENTE', 'monto_deuda', 'dias_vencido'], dtype='object')

##**3- Data Wrangling**##

###*3.1 Trasformación de headers de las columnas*####